In [8]:
# install required packages

import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import requests

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
!pip install lxml
!pip install html5lib
!pip install BeautifulSoup4

print('Folium installed')
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forge
    lib

In [9]:
import pandas as pd

In [10]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Seattle', attrs = {'class': 'wikitable'})
df = df[0]

In [11]:
df.head()

,Unnamed: 0,Neighborhood name,Within larger district,Annexed[41],Locator map,Street map,Image,Notes
0,1,North Seattle,Seattle,Various,NaN,NaN,NaN,North of the Lake Washington Ship Canal[42]
1,2,Broadview,North Seattle[42],1954[43],NaN,NaN,NaN,[44]
2,3,Bitter Lake,North Seattle[42],1954[43],NaN,NaN,NaN,[45]
3,4,North Beach / Blue Ridge,North Seattle[42],"1940,[43] 1954[43]",NaN,NaN,NaN,[46]
4,5,Crown Hill,North Seattle[42],"1907,[47] 1952,[43] 1954[43]",NaN,NaN,NaN,[48]


In [12]:
# df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)
df.drop(columns=['Unnamed: 0','Annexed[41]', 'Locator map', 'Street map', 'Image', 'Notes'], inplace=True)
print('data frame shape', df.shape)
df.head()

data frame shape (127, 2)


,Neighborhood name,Within larger district
0,North Seattle,Seattle
1,Broadview,North Seattle[42]
2,Bitter Lake,North Seattle[42]
3,North Beach / Blue Ridge,North Seattle[42]
4,Crown Hill,North Seattle[42]


In [13]:
df1 = df.rename(columns={"Within larger district": "District", 'Neighborhood name': 'Neighborhood'})
print('data frame shape', df1.shape)
df1.head()

data frame shape (127, 2)


,Neighborhood,District
0,North Seattle,Seattle
1,Broadview,North Seattle[42]
2,Bitter Lake,North Seattle[42]
3,North Beach / Blue Ridge,North Seattle[42]
4,Crown Hill,North Seattle[42]


In [14]:
df_population = pd.read_csv("Seattle_Neighborhood_population.csv", delimiter=",")
df_population = df_population.rename(columns={"neighborhoods": "Neighborhood"})
print('data frame shape', df_population.shape)
df_population.head()

data frame shape (127, 2)


,Neighborhood,population
0,Adams,112000
1,Alki Point,74400
2,Arbor Heights,54300
3,Atlantic,101200
4,Ballard,133900


In [15]:
neighborhoods = pd.merge(df1, df_population, on='Neighborhood')
print("Shapre of the data frame", neighborhoods.shape)
neighborhoods.head()

Shapre of the data frame (126, 3)


,Neighborhood,District,population
0,North Seattle,Seattle,73310
1,Broadview,North Seattle[42],79900
2,Bitter Lake,North Seattle[42],75000
3,North Beach / Blue Ridge,North Seattle[42],74380
4,Crown Hill,North Seattle[42],3100


In [16]:
# [^a-zA-Z 0-9 \n\.]', '')
neighborhoods['District'] = neighborhoods.District.str.replace(r'[^a-zA-Z ]', '')
neighborhoods['Neighborhood'] = neighborhoods.Neighborhood.str.replace(r'[^a-zA-Z ]', '')

neighborhoods.head()

,Neighborhood,District,population
0,North Seattle,Seattle,73310
1,Broadview,North Seattle,79900
2,Bitter Lake,North Seattle,75000
3,North Beach Blue Ridge,North Seattle,74380
4,Crown Hill,North Seattle,3100


In [17]:
# get the Geo locations from and save them in list

latitudeList = []
longitudeList = []

for value in neighborhoods['Neighborhood']:
    address = value + ', Seattle, US'
    try:
        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        latitudeList.append(latitude) 
        longitudeList.append(longitude)
    except:
        latitudeList.append('Not Found')
        longitudeList.append('Not Found')

In [18]:
neighborhoods['latitude'] = latitudeList
neighborhoods['longitude'] = longitudeList
neighborhoods.head()

,Neighborhood,District,population,latitude,longitude
0,North Seattle,Seattle,73310,47.6608,-122.291
1,Broadview,North Seattle,79900,47.7223,-122.36
2,Bitter Lake,North Seattle,75000,47.7262,-122.349
3,North Beach Blue Ridge,North Seattle,74380,Not Found,Not Found
4,Crown Hill,North Seattle,3100,47.6947,-122.371


In [19]:
# process the cells that have an Not Found location
neighborhoods.drop(neighborhoods[neighborhoods['longitude'] == 'Not Found'].index, inplace=True)
print("Shape of Data Frame - ", neighborhoods.shape)
neighborhoods.head()

Shape of Data Frame -  (102, 5)


,Neighborhood,District,population,latitude,longitude
0,North Seattle,Seattle,73310,47.6608,-122.291
1,Broadview,North Seattle,79900,47.7223,-122.36
2,Bitter Lake,North Seattle,75000,47.7262,-122.349
4,Crown Hill,North Seattle,3100,47.6947,-122.371
5,Greenwood,North Seattle,9800,47.691,-122.355


## Now we have perfect data to go ahead and start using Foursquare APIs to explore, visualize the existing Coffee Shops

### Define Foursquare Credentials and Version

##### Make sure that you have created a Foursquare developer account and have your credentials handy

In [20]:
CLIENT_ID = 'EZU3NNX0BFUXMY5KLVPGCZHL4V1BWKVDNNQ4SUSP3WNL0LQ1' # your Foursquare ID
CLIENT_SECRET = 'YFWMVYKKS14SQ1D2WZNM5TSJVTEUR4UNVJ4TFBOZ4TJLP4XZ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 200
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EZU3NNX0BFUXMY5KLVPGCZHL4V1BWKVDNNQ4SUSP3WNL0LQ1
CLIENT_SECRET:YFWMVYKKS14SQ1D2WZNM5TSJVTEUR4UNVJ4TFBOZ4TJLP4XZ


## 1. Search for a specific venue category
> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

In [21]:
search_query = 'coffee'
radius = 5000
print(search_query + ' .... OK!')

coffee .... OK!


#### Define the corresponding URL

In [22]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=EZU3NNX0BFUXMY5KLVPGCZHL4V1BWKVDNNQ4SUSP3WNL0LQ1&client_secret=YFWMVYKKS14SQ1D2WZNM5TSJVTEUR4UNVJ4TFBOZ4TJLP4XZ&ll=47.5384514,-122.3773869&v=20180604&query=coffee&radius=5000&limit=200'

#### Send the GET Request and examine the results

In [23]:
import requests

results = requests.get(url).json()
# results

#### Get relevant part of JSON and transform it into a *pandas* dataframe

In [24]:
# assign relevant part of JSON to venues
venues = results['response']['venues']
count = 0

for venue in venues:
    count = count + 1
    
print('Count ', count)
# tranform venues into a dataframe
dataframe = json_normalize(venues)
print("Shape of the data frame - ", dataframe.shape)
dataframe.head()

Count  50
Shape of the data frame -  (50, 24)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,45aaa8c6f964a5203b411fe3,C & P Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1580602850,False,5612 California Ave SW,at SW Findlay,47.551718,-122.386730,"[{'label': 'display', 'lat': 47.55171750304562...",...,WA,United States,"[5612 California Ave SW (at SW Findlay), Seatt...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,44170c45f964a5200b311fe3,All City Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1580602850,False,1205 S Vale St,at Airport Way S,47.549069,-122.317203,"[{'label': 'display', 'lat': 47.54906940796485...",...,WA,United States,"[1205 S Vale St (at Airport Way S), Seattle, W...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,49cc26ebf964a5204d591fe3,Cupcake Royale and Verite Coffee,"[{'id': '4bf58dd8d48988d1bc941735', 'name': 'C...",v-1580602850,False,4556 California Ave SW,at SW Alaska St,47.561185,-122.386668,"[{'label': 'display', 'lat': 47.56118494421977...",...,WA,United States,"[4556 California Ave SW (at SW Alaska St), Sea...",1452740,https://www.grubhub.com/restaurant/cupcake-roy...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN
3,5b0a1ea575eee4002c3329da,Onurka Applied Coffee Research Lab 3.0,"[{'id': '4bf58dd8d48988d155941735', 'name': 'G...",v-1580602850,False,NaN,NaN,47.537906,-122.369671,"[{'label': 'display', 'lat': 47.537906, 'lng':...",...,WA,United States,"[Seattle, WA 98126, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,49c56afbf964a520ff561fe3,Freshy's Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1580602850,False,2735 California Ave SW,SW Lander St,47.578201,-122.386666,"[{'label': 'display', 'lat': 47.5782012454133,...",...,WA,United States,"[2735 California Ave SW (SW Lander St), Seattl...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Define information of interest and filter dataframe

In [25]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,C & P Coffee,Coffee Shop,5612 California Ave SW,at SW Findlay,47.551718,-122.386730,"[{'label': 'display', 'lat': 47.55171750304562...",1635,98136,US,Seattle,WA,United States,"[5612 California Ave SW (at SW Findlay), Seatt...",45aaa8c6f964a5203b411fe3
1,All City Coffee,Coffee Shop,1205 S Vale St,at Airport Way S,47.549069,-122.317203,"[{'label': 'display', 'lat': 47.54906940796485...",4674,98108,US,Seattle,WA,United States,"[1205 S Vale St (at Airport Way S), Seattle, W...",44170c45f964a5200b311fe3
2,Cupcake Royale and Verite Coffee,Cupcake Shop,4556 California Ave SW,at SW Alaska St,47.561185,-122.386668,"[{'label': 'display', 'lat': 47.56118494421977...",2624,98116,US,Seattle,WA,United States,"[4556 California Ave SW (at SW Alaska St), Sea...",49cc26ebf964a5204d591fe3
3,Onurka Applied Coffee Research Lab 3.0,Gastropub,NaN,NaN,47.537906,-122.369671,"[{'label': 'display', 'lat': 47.537906, 'lng':...",583,98126,US,Seattle,WA,United States,"[Seattle, WA 98126, United States]",5b0a1ea575eee4002c3329da
4,Freshy's Coffee,Coffee Shop,2735 California Ave SW,SW Lander St,47.578201,-122.386666,"[{'label': 'display', 'lat': 47.5782012454133,...",4479,98116,US,Seattle,WA,United States,"[2735 California Ave SW (SW Lander St), Seattl...",49c56afbf964a520ff561fe3
5,Youngstown Coffee Company,Coffee Shop,6032 California Ave SW,NaN,47.547419,-122.387006,"[{'label': 'display', 'lat': 47.547419, 'lng':...",1232,98136,US,Seattle,WA,United States,"[6032 California Ave SW, Seattle, WA 98136, Un...",5d1cde6f7950f70023091d9c
6,Tully's Coffee,Coffee Shop,4201 SW Morgan St,NaN,47.544701,-122.386002,"[{'label': 'display', 'lat': 47.544701, 'lng':...",950,98136,US,Seattle,WA,United States,"[4201 SW Morgan St, Seattle, WA 98136, United ...",4f328a0819836c91c7e1ad5f
7,Coffee Of The Islands 9th Fl,Conference Room,9th floor,NaN,47.581163,-122.335744,"[{'label': 'display', 'lat': 47.581163, 'lng':...",5691,98134,US,Seattle,WA,United States,"[9th floor, Seattle, WA 98134, United States]",50bf933ae4b04651f7a6a256
8,WS Coffee House,Coffee Shop,NaN,NaN,47.548310,-122.376192,"[{'label': 'display', 'lat': 47.54831030623475...",1101,NaN,US,NaN,Washington,United States,"[Washington, United States]",4e6103db52b1260c127e5c99
9,Bedhead Coffee,Coffee Shop,NaN,NaN,47.533482,-122.354827,"[{'label': 'display', 'lat': 47.533482, 'lng':...",1783,98106,US,Seattle,WA,United States,"[Seattle, WA 98106, United States]",5d82700095ffc6000768a88e


#### Let's visualize the Italian restaurants that are nearby

In [26]:
dataframe_filtered.name

0                                          C & P Coffee
1                                       All City Coffee
2                      Cupcake Royale and Verite Coffee
3                Onurka Applied Coffee Research Lab 3.0
4                                       Freshy's Coffee
5                             Youngstown Coffee Company
6                                        Tully's Coffee
7                          Coffee Of The Islands 9th Fl
8                                       WS Coffee House
9                                        Bedhead Coffee
10                                           QED Coffee
11                                      Moonshot Coffee
12                                       Tully's Coffee
13                            Youngstown Coffee Company
14                          Geno's Coffee Shop & Bakery
15                                       Olympia Coffee
16                                      Realfine Coffee
17                                            St

In [27]:
import csv

f = open("my_csv.csv", "w")
writer = csv.DictWriter(
    f, fieldnames=['name', 'categories', 'address', 'crossStreet', 'lat', 'lng', 'labeledLatLngs', 'distance', 'postalCode', 'cc', 'city', 'state', 'country', 'formattedAddress', 'id', 'neighborhood'])
writer.writeheader()
f.close()


In [28]:
# Foursquare developer account and credentials

CLIENT_ID = 'EZU3NNX0BFUXMY5KLVPGCZHL4V1BWKVDNNQ4SUSP3WNL0LQ1' # your Foursquare ID
CLIENT_SECRET = 'YFWMVYKKS14SQ1D2WZNM5TSJVTEUR4UNVJ4TFBOZ4TJLP4XZ' # your Foursquare Secret
VERSION = '20180604'
search_query = 'coffee'

def getCoffeeShops(CLIENT_ID, CLIENT_SECRET, search_query, latitude, longitude, neighborhood):
    LIMIT = 50
    radius = 5000
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()

    # assign relevant part of JSON to venues
    venues = results['response']['venues']

    # tranform venues into a dataframe
    dataframe = json_normalize(venues)

    # keep only columns that include venue name, and anything that is associated with location
    filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]

    # function that extracts the category of the venue
    def get_category_type(row):
        try:
            categories_list = row['categories']
        except:
            categories_list = row['venue.categories']

        if len(categories_list) == 0:
            return None
        else:
            return categories_list[0]['name']

    # filter the category for each row
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

    # clean column names by keeping only last term
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
    dataframe_filtered['Neighborhood'] = neighborhood
    dataframe_filtered.to_csv('my_csv.csv', mode='a', header=False)
    
    return dataframe_filtered.count()

In [29]:
# Get list of coffeee shops in each neighborhood. 
# Since the data is going to be so much and many requests to get the data from Four Square, we are going to save the dataframe for and continue using that in the next stops
# totalCoffeeShops = []

# for index, neighborhood in neighborhoods.iterrows():
#     count = 
#     totalCoffeeShops.append(getCoffeeShops(CLIENT_ID, CLIENT_SECRET, search_query, latitude=neighborhood['latitude'], longitude=neighborhood['longitude'], neighborhood=neighborhood['Neighborhood']))

# totalCoffeeShops

In [33]:
CLIENT_ID = 'EZU3NNX0BFUXMY5KLVPGCZHL4V1BWKVDNNQ4SUSP3WNL0LQ1' # your Foursquare ID
CLIENT_SECRET = 'YFWMVYKKS14SQ1D2WZNM5TSJVTEUR4UNVJ4TFBOZ4TJLP4XZ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50 # FourSquare search limits sending max of 50 venues
search_query = 'Coffee Shop'
radius = 2000

def getCoffeeShops(CLIENT_ID, CLIENT_SECRET, search_query, latitude, longitude, neighborhood):
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

    results = requests.get(url).json()
    venues = results['response']['venues']
    count = 0

    for venue in venues:
        count = count + 1
        
    return count

In [34]:
# Get Number of coffeee shops in each neighborhood. 
# Since the data is going to be so much and many requests to get the data from Four Square, we are going to save the dataframe for and continue using that in the next stops

totalCoffeeShops = []
for index, neighborhood in neighborhoods.iterrows():
    count = getCoffeeShops(CLIENT_ID, CLIENT_SECRET, search_query, latitude=neighborhood['latitude'], longitude=neighborhood['longitude'], neighborhood=neighborhood['Neighborhood'])
    totalCoffeeShops.append(count)

totalCoffeeShops

[50,
 18,
 20,
 45,
 47,
 39,
 28,
 50,
 45,
 19,
 12,
 13,
 18,
 20,
 31,
 25,
 21,
 9,
 50,
 50,
 34,
 10,
 25,
 21,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 22,
 50,
 35,
 50,
 24,
 50,
 21,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 12,
 22,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 38,
 50,
 27,
 20,
 25,
 21,
 20,
 17,
 45,
 32,
 50,
 19,
 27,
 17,
 25,
 4,
 22,
 28,
 14,
 7,
 12,
 31,
 31,
 33,
 19,
 21,
 31,
 15,
 16]

In [36]:
neighborhoods['totalCoffeeShops'] = totalCoffeeShops
neighborhoods.head()

,Neighborhood,District,population,latitude,longitude,totalCoffeeShops
0,North Seattle,Seattle,73310,47.6608,-122.291,50
1,Broadview,North Seattle,79900,47.7223,-122.36,18
2,Bitter Lake,North Seattle,75000,47.7262,-122.349,20
4,Crown Hill,North Seattle,3100,47.6947,-122.371,45
5,Greenwood,North Seattle,9800,47.691,-122.355,47


In [37]:
neighborhoods.describe()

,population,totalCoffeeShops
count,102.000000,102.000000
mean,78995.588235,36.303922
std,38865.383655,15.039742
min,3100.000000,4.000000
25%,66732.500000,21.000000
50%,73630.000000,45.000000
75%,79470.000000,50.000000
max,293400.000000,50.000000


In [38]:
neighborhoods.corr()

,population,totalCoffeeShops
population,1.000000,-0.061504
totalCoffeeShops,-0.061504,1.000000


In [60]:
neighborhoods['avg'] = neighborhoods.mean(axis=1)
neighborhoods.sort_values(by=['totalCoffeeShops'])

,Neighborhood,District,population,latitude,longitude,totalCoffeeShops,avg,popluationCoffeeAvg
109,Alki Point,West Seattle,74400,47.5763,-122.421,4,32551.500000,18600.000000
116,Fauntleroy,West Seattle,110900,47.5224,-122.393,7,45550.839286,15842.857143
18,Sand Point,North Seattle,69030,47.6824,-122.264,9,27807.125000,7670.000000
22,Windermere,North Seattle,63690,47.6709,-122.262,10,25479.750000,6369.000000
58,Madison Park,Capitol Hill Central Seattle,77370,47.6359,-122.28,12,30630.125000,6447.500000
...,...,...,...,...,...,...,...,...
49,Capitol Hill,Central Seattle,71700,47.619,-122.32,50,27264.750000,1434.000000
48,West Queen Anne,Queen Anne,64760,47.6395,-122.361,50,24627.550000,1295.200000
47,Lower Queen Anne,Queen Anne,77800,47.6223,-122.356,50,29582.750000,1556.000000
45,North Queen Anne,Queen Anne,73520,47.6395,-122.361,50,27956.350000,1470.400000


In [59]:
neighborhoods1 = neighborhoods
neighborhoods1['popluationCoffeeAvg'] = neighborhoods1[' population']/neighborhoods1['totalCoffeeShops']
neighborhoods1.sort_values(['popluationCoffeeAvg'])

,Neighborhood,District,population,latitude,longitude,totalCoffeeShops,avg,popluationCoffeeAvg
4,Crown Hill,North Seattle,3100,47.6947,-122.371,45,1572.5,68.888889
5,Greenwood,North Seattle,9800,47.691,-122.355,47,4923.5,208.510638
41,Briarcliff,Magnolia,7400,47.6482,-122.408,21,3710.5,352.380952
90,Columbia City,Rainier Valley,10900,47.5579,-122.285,25,5462.5,436.000000
51,Broadway,Capitol Hill,30800,47.6091,-122.321,50,15425.0,616.000000
...,...,...,...,...,...,...,...,...
7,Haller Lake,Northgate,270100,47.7197,-122.334,28,135064.0,9646.428571
11,Cedar Park,Lake City,117300,47.7257,-122.286,12,58656.0,9775.000000
59,Broadmoor,Madison Park,293400,47.6352,-122.29,22,146711.0,13336.363636
116,Fauntleroy,West Seattle,110900,47.5224,-122.393,7,55453.5,15842.857143


In [68]:
# potential prespect / opportunity to open Coffee Shop

ptCoffee = neighborhoods1.loc[neighborhoods1['Neighborhood'] == 'Alki Point']
latitude = ptCoffee['latitude'].values[0]
longitude = ptCoffee['longitude'].values[0]
ptCoffee

,Neighborhood,District,population,latitude,longitude,totalCoffeeShops,avg,popluationCoffeeAvg
109,Alki Point,West Seattle,74400,47.5763,-122.421,4,32551.5,18600.0


In [69]:
# Let us GET and load the existing venues from Four Sqaure and map the opportunity location
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=EZU3NNX0BFUXMY5KLVPGCZHL4V1BWKVDNNQ4SUSP3WNL0LQ1&client_secret=YFWMVYKKS14SQ1D2WZNM5TSJVTEUR4UNVJ4TFBOZ4TJLP4XZ&ll=47.5762895,-122.4207968&v=20180604&query=Coffee Shop&radius=2000&limit=50'

In [70]:
results = requests.get(url).json()
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
print("Shape of the data frame - ", dataframe.shape)
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

Shape of the data frame -  (4, 24)


In [71]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map